# CS 1656 – Introduction to Data Science 

## Instructor: Alexandros Labrinidis / Teaching Assistant: Tahereh Arabghalizi
### Additional credits: Evangelos Karageorgos, Zuha Agha, Anatoli Shein, Phuong Pham
## Lab 09: SQL via Data API
---
In this lab, you will execute SQL queries on real data by connecting to the open data portal of [Western Pennsylavnia Regional Data Center](https://www.wprdc.org/) and requesting data via API calls.  

In [2]:
import json
from datetime import datetime, timedelta, date
import requests
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

We will be using Allegheny County Restaurant/Food Facility Inspection Violation Dataset found here https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations. This dataset contains violation data from actual routine inspections by one of health department staff's members for the last two years. It should be fun to find out inspection results for places where we eat in Pittsburgh! =)

In [7]:
wprdc_api_endpoint = "https://data.wprdc.org/api/3/action/datastore_search_sql"

# id for database table
resource_id = "1a1329e2-418c-4bd3-af2c-cc334e7559af"

# Get the date from 90 days ago)
start_date = datetime.now() - timedelta(days=180)

# Convert to a string the format the the data center accepts (yyyy-mm-dd)
start_str = start_date.strftime("%Y-%m-%d")

# SQL query we'll use in API call to request data
query = """
SELECT *
FROM "{}"
WHERE "INSPECT_DT" >= '{}' AND "CITY" = '{}'
;""".format(resource_id, start_str, "Pittsburgh")

# Make WPRDC API Call
response = requests.get(wprdc_api_endpoint, {'sql': query})

# Parse response JSON into python dictionary
response_data = json.loads(response.text)
#print(response_data) 

# Convert dictionary to dataframe
df = pd.DataFrame.from_dict(response_data['result']['records'])

# Print the number of rows
print(df.shape[0], "rows total")

df.head()

6130 rows total


,BUS_ST_DATE,CITY,DESCRIPTION,DESCRIPTION_NEW,ENCOUNTER,END_TIME,FACILITY_NAME,HIGH,ID,INSPECT_DT,...,NUM,PLACARD_ST,RATING,START_TIME,STATE,STREET,URL,ZIP,_full_text,_id
0,1996-11-22T00:00:00,Pittsburgh,Restaurant with Liquor,Ventilation,201806210019,2018-08-08T15:00:00,Walker's Pub,,56024,2018-06-21T00:00:00,...,2024,1,V,2018-08-08T13:45:00,PA,Sarah St,http://appsrv.achd.net/reports/rwservlet?food_...,15203,"'-06':15 '-08':22,23,29,30 '-11':3 '-117':46 '...",177759
1,2017-10-31T00:00:00,Pittsburgh,Chain Restaurant with Liquor,Cold Holding Temperatures,201805230031,2018-08-08T14:00:00,Joe's Crab Shack,T,201711140005,2018-05-23T00:00:00,...,226,1,V,2018-08-08T13:05:00,PA,W Station Square Drive,http://appsrv.achd.net/reports/rwservlet?food_...,15219,"'-05':13 '-08':30,31,44,45 '-10':37 '-119':50 ...",176001
2,2017-10-31T00:00:00,Pittsburgh,Chain Restaurant with Liquor,Facilities to Maintain Temperature,201805230031,2018-08-08T14:00:00,Joe's Crab Shack,F,201711140005,2018-05-23T00:00:00,...,226,1,V,2018-08-08T13:05:00,PA,W Station Square Drive,http://appsrv.achd.net/reports/rwservlet?food_...,15219,"'-05':13 '-08':31,32,45,46 '-10':38 '-119':51 ...",176002
3,1997-07-10T00:00:00,Pittsburgh,Chain Restaurant without Liquor,Cold Holding Temperatures,201805230032,2018-08-08T14:00:00,Arby's #1854,T,56609,2018-05-23T00:00:00,...,1911,1,V,2018-08-08T13:05:00,PA,Braddock Avenue,http://appsrv.achd.net/reports/rwservlet?food_...,15218,"'-05':11 '-07':25 '-08':33,34,39,40 '-10':26 '...",176003
4,2018-03-23T00:00:00,Pittsburgh,Rooming House with Guest food Service,Date Marking of Food,201805240008,2018-08-08T09:10:00,Drury Inn Airport,F,201804030002,2018-05-24T00:00:00,...,101,1,V,2018-08-08T08:30:00,PA,Ridge Road,http://appsrv.achd.net/reports/rwservlet?food_...,15205,"'-03':18 '-05':3 '-08':25,26,47,48 '-23':19 '-...",176251


Details of dataset attributes are below. (Taken from https://data.wprdc.org/dataset/allegheny-county-restaurant-food-facility-inspection-violations/resource/1a1329e2-418c-4bd3-af2c-cc334e7559af)


## Queries

__Q1) Find all unique decription categories of violation in Pittsburgh restaurants over the past six months.__

In [8]:
query = """
SELECT DISTINCT "DESCRIPTION_NEW"
FROM "{}"
WHERE "INSPECT_DT" >= '{}' AND "CITY" = '{}'
;""".format(resource_id, start_str, "Pittsburgh")

response = requests.get(wprdc_api_endpoint, {'sql': query})

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

,DESCRIPTION_NEW
0,Employee Health
1,Lighting
2,General Premises
3,Cold Holding Temperatures
4,Toilet Room
5,Consumer Advisory
6,Garbage and Refuse
7,Toxic Items
8,Cleaning and Sanitization
9,Demonstration of Knowledge


__Q2) Find restaurants in Pittsburgh with no violations in at least one decription category over the past six months.__

In [9]:
query = """
SELECT "FACILITY_NAME", COUNT("FACILITY_NAME")
FROM "{}"
WHERE "INSPECT_DT" >= '{}' AND "CITY" = '{}' AND "RATING" <> '{}'
GROUP BY "FACILITY_NAME"
;""".format(resource_id, start_str, "Pittsburgh", "V")

response = requests.get(wprdc_api_endpoint, {'sql': query})

df = pd.DataFrame.from_dict(json.loads(response.text)['result']['records'])

df

,FACILITY_NAME,count
0,Aldi #56,1
1,Alquisiras Paleteria,1
2,Au Bon Pain #103 @ US Steel Tower Concourse,1
3,Bea Taco Town #1,1
4,Bethlehem Haven of Pittsburgh Rooms,1
5,BFG Caf,1
6,Brookline N Sinker Bar,1
7,Cains Local Bar & Kitchen,3
8,Chameleon Concessions Commissary,1
9,Chateau Cafe & Cakery,1


### Tasks

__T1) Find top 30 restaurants in Pittsburgh with maximum number of violations.__

Lets look more closely into the inspection results of Pitt facilities.

__T2) Find the category descriptions and their high, medium, low risk ratings for all violations at facilities that start with 'Pitt' over the past six months.__

Now lets look at all facilities that contain word 'Pitt'.

__T3) Find the category descriptions and their high, medium, low risk ratings for all violations at all facilities that have word 'Pitt' in their name over the past six months. Note that results that contain word 'Pitt' as part of another word (e.g. 'Pittsburgh') should not be included__

__T4) Find top 20 facilities that have word 'Pitt' in their name and have the highest counts of violations over the past six months.__

Let's make a bar graph of these counts now

__T5) Create a bar graph of the counts from T4).__